In [1]:
# pip install selenium-wire
from seleniumwire import webdriver  # <— note seleniumwire instead of selenium
from selenium.webdriver.common.by import By
import time
import json

TOKEN_URL = "https://swisstennisch.b2clogin.com/swisstennisch.onmicrosoft.com/" \
            "b2c_1a_b2c_1_signup-signin/oauth2/v2.0/token"

driver = webdriver.Chrome()

try:
    # 1) Log in as before:
    driver.get("https://www.mytennis.ch/fr")
    time.sleep(2)
    driver.find_element(By.ID, "UserId").send_keys("benoitphilippe")
    driver.find_element(By.ID, "password").send_keys("Tennisman14.")
    driver.find_element(By.ID, "next").click()
    time.sleep(10)

    # 2) Find the token request and print its JSON response
    for req in driver.requests:
        if req.method == "POST" and req.url.startswith(TOKEN_URL):
            # wait until the server has sent a response
            if req.response:
                body = req.response.body.decode("utf-8")
                token_data = json.loads(body)
                print("Status:", req.response.status_code)
                print("JSON response:")
                print(json.dumps(token_data, indent=2))
                break
    else:
        print("No token request found—maybe it hasn't fired yet.")
finally:
    driver.quit()


Status: 200
JSON response:
{
  "id_token": "eyJhbGciOiJSUzI1NiIsImtpZCI6Il96XzBaVG1qVC1vS3NyWkdtLWJ3Q0piUnphcGlWcXBwNWhLUmJwU2ZzZFUiLCJ0eXAiOiJKV1QifQ.eyJleHAiOjE3NDg0NDgwMzcsIm5iZiI6MTc0ODQ0NDQzNywidmVyIjoiMS4wIiwiaXNzIjoiaHR0cHM6Ly9zd2lzc3Rlbm5pc2NoLmIyY2xvZ2luLmNvbS84Yjk2NmEyZC02NmNhLTQzMjgtOTAxNS1kYWQxYWFiNmQxNjQvdjIuMC8iLCJzdWIiOiIyZmFjZTNiZi03ZDNmLTQ5MDEtOTM1OC0xNDU2ODc0YzE5OTMiLCJhdWQiOiI1YzhjNmU0ZS1hMDE3LTQ5ZDQtYTljMC1mODlmZTA4NjE0ZjMiLCJub25jZSI6IjQxZDhhODRiLTIzYjYtNDgzZC1iNmQzLTdjYzc2MDhkNWY1YiIsImlhdCI6MTc0ODQ0NDQzNywiYXV0aF90aW1lIjoxNzQ4NDQ0NDM0LCJleHRlbnNpb25fT3JkaW5hcnlFbWFpbCI6Imxlb2RhZ2FuZmVkZXJlckBob3RtYWlsLmZyIiwiZ2l2ZW5fbmFtZSI6IlBoaWxpcHBlIiwiZmFtaWx5X25hbWUiOiJCZW5vaXQiLCJleHRlbnNpb25fR2VuZGVyIjoibWFsZSIsImV4dGVuc2lvbl9MYW5ndWFnZSI6ImZyIiwiZXh0ZW5zaW9uX1BlcnNvbk5yIjoiMTk4NjA2NTYiLCJleHRlbnNpb25fUm9sZXMiOiIiLCJleHRlbnNpb25fRmF2b3JpdGVzIjoiIiwiZXh0ZW5zaW9uX0Fzc29jaWF0ZVR5cGVJRCI6IiIsInRmcCI6IkIyQ18xQV9CMkNfMV9zaWdudXAtc2lnbmluIn0.UO0G-iH3Gt1TkU27YyQGvuLJE-TUOj4qDdGdS

# get all players

In [2]:

import requests

url = "https://hasura.swisstennis.ch/v1/graphql"
headers = {"authorization": f"Bearer {token_data['id_token']}", "content-type": "application/json"}


In [3]:
payload = {
    "query":
    "query PlayerList($offset: Int, $limit: Int, $id: Int, $order_by: [lizenz_nehmer_order_by!], $where: lizenz_nehmer_bool_exp) {\n  lizenz_nehmer(\n    offset: $offset\n    limit: $limit\n    where: $where\n    order_by: $order_by\n  ) {\n    licenceNumber\n    classification\n    classificationValue\n    competitionValue\n    lastRanking\n    ageCategoryRedundant\n    currentLicenceStatusId\n    ranking\n    person {\n      id\n      firstname\n      lastname\n    }\n  }\n  user_profile_frienship(where: {personId: {_eq: $id}}) {\n    playerId\n  }\n}",
    "variables": {
        "id": 19860656,
        "offset": 0,
        "limit": 2000,
        "where": {
            "kontingent": {
                "_eq": 1
            },
            "currentLicenceStatusId": {
                "_lt": 3
            },
            "person": {
                "gender": {
                    "_eq": 1
                }
            },
        },
        "order_by": [{
            "classificationValue": "desc"
        }, {
            "ranking": "asc"
        }]
    },
    "operationName": "PlayerList"
}

response = requests.post(url, json=payload, headers=headers)
print(response.status_code)


200


In [4]:
data= response.json()
data

{'data': {'lizenz_nehmer': [{'licenceNumber': '936.85.190.0',
    'classification': 'N1',
    'classificationValue': 16.39,
    'competitionValue': 15.299,
    'lastRanking': 5,
    'ageCategoryRedundant': '40+',
    'currentLicenceStatusId': 2,
    'ranking': 1,
    'person': {'id': 70318, 'firstname': 'Stan', 'lastname': 'Wawrinka'}},
   {'licenceNumber': '497.96.286.0',
    'classification': 'N1',
    'classificationValue': 15.945,
    'competitionValue': 14.816,
    'lastRanking': 3,
    'ageCategoryRedundant': 'A',
    'currentLicenceStatusId': 1,
    'ranking': 2,
    'person': {'id': 126520,
     'firstname': 'Marc-Andrea',
     'lastname': 'Hüsler'}},
   {'licenceNumber': '574.03.143.0',
    'classification': 'N1',
    'classificationValue': 15.787,
    'competitionValue': 14.604,
    'lastRanking': 6,
    'ageCategoryRedundant': 'A',
    'currentLicenceStatusId': 1,
    'ranking': 3,
    'person': {'id': 172242, 'firstname': 'Jérôme', 'lastname': 'Kym'}},
   {'licenceNumber': 

In [5]:
import pandas as pd


df = pd.json_normalize(data['data']['lizenz_nehmer'])

# Create 'name' column by combining first and last names
df['name'] = df['person.firstname'] + ' ' + df['person.lastname']
df.columns=[c.split("person.")[-1] for c in df.columns]
df=df[['id','name',"competitionValue"]]
df.to_parquet("data/all_players.parquet")


In [6]:
df[df['name']=="Leonard Ung"]

,id,name,competitionValue
993,134639,Leonard Ung,6.967


# each player

In [7]:
import pandas as pd
all_players = pd.read_parquet("data/all_players.parquet")
print(all_players)

            id                 name  competitionValue
0        70318        Stan Wawrinka            15.299
1       126520   Marc-Andrea Hüsler            14.816
2       172242           Jérôme Kym            14.604
3       122182  Alexander Ritschard            14.644
4       158613     Dominic Stricker            14.558
...        ...                  ...               ...
1995    180583           Remo Fuchs             5.726
1996  19810106         Lukas Würsch             5.708
1997    177482      Reto Eisenegger             5.570
1998    113706        Lucas Schibli             5.912
1999    195416          Calvin Kläy             5.952

[2000 rows x 3 columns]


In [90]:
from tqdm import tqdm
import requests
import os

i = 0
os.makedirs("data/singles", exist_ok=True)
for player in tqdm(all_players.itertuples(), total=len(all_players)):
    # if player.name!="Severin Habe":
    #     continue
    i += 1
    if i < 1000:
        continue
    payload = {
        "query":
        "query playerRankingHistory($id: Int!, $yearFrom: timestamp, $yearTo: timestamp) {\n  list: RankingHistory(\n    where: {personId: {_eq: $id}, date: {_lte: $yearTo, _gte: $yearFrom}}\n    order_by: {date: desc}\n  ) {\n    personId\n    date\n    classification\n    rank\n    competitionValue\n    classificationValue\n    games\n  }\n  currentRanking: lizenz_nehmer(where: {personId: {_eq: $id}}) {\n    ranking\n    classification\n    classificationValue\n    competitionValue\n    numberOfGames\n  }\n  currentSeason: RankSeasonRange(where: {seasonPointer: {_eq: 1}}) {\n    dateBegin\n  }\n  previousSeason: RankSeasonRange(where: {seasonPointer: {_eq: 0}}) {\n    dateBegin\n  }\n}",
        "variables": {
            "id": player.id,
            "yearFrom": "2018-12-31T23:00:00.000Z",
            "yearTo": "2025-12-31T22:59:59.999Z"
        },
        "operationName": "playerRankingHistory"
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()["data"]["list"]
    if len(data) == 0:
        continue
    competitionValue = data[0]["competitionValue"]

    payload = {
        "query":
        "query SingleResults($where: AllSingleResults_bool_exp, $orderBy: [AllSingleResults_order_by!], $limit: Int = 300) {\n  results: AllSingleResults(where: $where, order_by: $orderBy, limit: $limit) {\n    ...singleResult\n  }\n}\n\nfragment singleResult on AllSingleResults {\n  date\n  tournamentName\n  adversaryPersonId\n  adversaryFirstname\n  adversaryLastname\n  playerSet1WonGames\n  adversarySet1WonGames\n  playerSet2WonGames\n  adversarySet2WonGames\n  playerSet3WonGames\n  adversarySet3WonGames\n  playerSet4WonGames\n  adversarySet4WonGames\n  playerSet5WonGames\n  adversarySet5WonGames\n  playerWinnerCode\n  Asr_AdversaryValue_f\n  adversary {\n    classification\n    classificationValue\n    competitionValue4L\n    ranking\n  }\n  ATPWTARank\n  Asr_Id_Item_l\n  round: Asr_Round_i\n  encounterId: Asr_Id_Item_l\n  matchNotConsidered\n  event {\n    id\n    tournamentId\n    eventModeId\n  }\n  source\n  rank: Asr_RankConvert_s\n}",
        "variables": {
            "where": {
                "playerPersonId": {
                    "_eq": player.id
                },
                "playerWinnerCode": {
                    "_neq": "D"
                },
                "source": {
                    "_in": [
                        310, 311, 110, 111, 120, 121, 210, 211, 220, 221, 250, 150, 410, 411, 420, 421, 450, 500, 501,
                        600, 601, 660, 610, 650, 700, 701
                    ]
                },
                "rankSeasonRange": {
                    "seasonPointer": {
                        "_in": [-1, 0]
                    }
                }
            },
            "orderBy": {
                "date": "desc"
            }
        },
        "operationName": "SingleResults"
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()["data"]["results"]
    df = pd.json_normalize(data)
    if len(df) == 0:
        df.to_parquet(
            f"data/singles/{player.name.replace(' ','*')}_competitionValuePrev={competitionValue}__competitionValue={player.competitionValue}.parquet"
        )
        continue
    wanted = [
        "date",
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "rank",
        "adversary.competitionValue4L",
        "Asr_AdversaryValue_f",
        "adversaryPersonId",
    ]
    existing = [col for col in wanted if col in df.columns]
    df = df[existing]

    df["date"] = pd.to_datetime(df["date"], format='mixed')
    df = df.set_index("date").sort_index(ascending=False)
    if "adversary.classification" in df.columns:
        df["adversary.classification"] = df["adversary.classification"].fillna(df["rank"])
    if "adversary.competitionValue4L" not in df.columns:
        df["adversary.competitionValue4L"] = df["Asr_AdversaryValue_f"]
    else:
        df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(df["Asr_AdversaryValue_f"])

    wanted = [
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "adversary.competitionValue4L",
        "adversaryPersonId",
    ]
    existing = [col for col in wanted if col in df.columns]
    df = df[existing]
    df = df[df["adversary.competitionValue4L"] != 0]
    new_player=player
    df.to_parquet(
        f"data/singles/{player.name.replace(' ','*')}_competitionValuePrev={competitionValue}__competitionValue={player.competitionValue}.parquet"
    )
    break
# df


  0%|          | 0/2000 [00:00<?, ?it/s]

 50%|████▉     | 999/2000 [00:00<00:00, 1531.53it/s]


In [92]:
import os
import requests
from tqdm import tqdm
import pandas as pd

missing_ids: set[int] = set()

os.makedirs("data/singles", exist_ok=True)

for player in tqdm(all_players.itertuples(), total=len(all_players)):
    payload = {
        "query": 
        "query playerRankingHistory($id: Int!, $yearFrom: timestamp, $yearTo: timestamp) {\n  list: RankingHistory(\n    where: {personId: {_eq: $id}, date: {_lte: $yearTo, _gte: $yearFrom}}\n    order_by: {date: desc}\n  ) {\n    personId\n    date\n    classification\n    rank\n    competitionValue\n    classificationValue\n    games\n  }\n  currentRanking: lizenz_nehmer(where: {personId: {_eq: $id}}) {\n    ranking\n    classification\n    classificationValue\n    competitionValue\n    numberOfGames\n  }\n  currentSeason: RankSeasonRange(where: {seasonPointer: {_eq: 1}}) {\n    dateBegin\n  }\n  previousSeason: RankSeasonRange(where: {seasonPointer: {_eq: 0}}) {\n    dateBegin\n  }\n}",
        "variables": {
            "id": player.id,
            "yearFrom": "2018-12-31T23:00:00.000Z",
            "yearTo":   "2025-12-31T22:59:59.999Z",
        },
        "operationName": "playerRankingHistory",
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()["data"]["list"]
    if len(data) == 0:
        continue
    competitionValue = data[0]["competitionValue"]

    payload = {
        "query": 
        "query SingleResults($where: AllSingleResults_bool_exp, $orderBy: [AllSingleResults_order_by!], $limit: Int = 300) {\n  results: AllSingleResults(where: $where, order_by: $orderBy, limit: $limit) {\n    ...singleResult\n  }\n}\n\nfragment singleResult on AllSingleResults {\n  date\n  tournamentName\n  adversaryPersonId\n  adversaryFirstname\n  adversaryLastname\n  playerSet1WonGames\n  adversarySet1WonGames\n  playerSet2WonGames\n  adversarySet2WonGames\n  playerSet3WonGames\n  adversarySet3WonGames\n  playerSet4WonGames\n  adversarySet4WonGames\n  playerSet5WonGames\n  adversarySet5WonGames\n  playerWinnerCode\n  Asr_AdversaryValue_f\n  adversary {\n    classification\n    classificationValue\n    competitionValue4L\n    ranking\n  }\n  ATPWTARank\n  Asr_Id_Item_l\n  round: Asr_Round_i\n  encounterId: Asr_Id_Item_l\n  matchNotConsidered\n  event {\n    id\n    tournamentId\n    eventModeId\n  }\n  source\n  rank: Asr_RankConvert_s\n}",
        "variables": {
            "where": {
                "playerPersonId": {"_eq": player.id},
                "playerWinnerCode": {"_neq": "D"},
                "source": {"_in": [
                    310, 311, 110, 111, 120, 121, 210, 211, 220, 221, 250, 150,
                    410, 411, 420, 421, 450, 500, 501, 600, 601, 660, 610, 650,
                    700, 701
                ]},
                "rankSeasonRange": {"seasonPointer": {"_in": [-1, 0]}},
            },
            "orderBy": {"date": "desc"},
        },
        "operationName": "SingleResults",
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()["data"]["results"]
    df = pd.json_normalize(data)

    # save empty → continue
    if len(df) == 0:
        df.to_parquet(
            f"data/singles/{player.name.replace(' ','*')}"
            f"_competitionValuePrev={competitionValue}"
            f"__competitionValue={player.competitionValue}.parquet"
        )
        continue

    wanted = [
        "date",
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "rank",
        "adversary.competitionValue4L",
        "Asr_AdversaryValue_f",
        "adversaryPersonId",
    ]
    df = df[[c for c in wanted if c in df.columns]]

    df["date"] = pd.to_datetime(df["date"], format="mixed")
    df = df.set_index("date").sort_index(ascending=False)

    if "adversary.classification" in df.columns:
        df["adversary.classification"] = (
            df["adversary.classification"].fillna(df["rank"])
        )
    if "adversary.competitionValue4L" not in df.columns:
        df["adversary.competitionValue4L"] = df["Asr_AdversaryValue_f"]
    else:
        df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(
            df["Asr_AdversaryValue_f"]
        )

    wanted = [
        "adversaryFirstname",
        "adversaryLastname",
        "playerWinnerCode",
        "adversary.classification",
        "adversary.competitionValue4L",
        "adversaryPersonId",
    ]
    df = df[[c for c in wanted if c in df.columns]]
    df = df[df["adversary.competitionValue4L"] != 0]

    if "adversaryPersonId" in df.columns:
        missing_ids.update(
            set(df["adversaryPersonId"].dropna().astype(int))
            - set(all_players["id"].astype(int))
        )

    # save results
    df.to_parquet(
        f"data/singles/{player.name.replace(' ','*')}"
        f"_competitionValuePrev={competitionValue}"
        f"__competitionValue={player.competitionValue}.parquet"
    )


  0%|          | 0/2000 [00:00<?, ?it/s]

  1%|          | 24/2000 [00:12<16:49,  1.96it/s]


KeyboardInterrupt: 

In [ ]:

# ── NEW: fetch rankingHistory for every missing adversary and append ──────────
for adv_id in tqdm(missing_ids, desc="Fetching missing adversaries"):
    payload = {
        "query": "...",
        "variables": {
            "id": adv_id,
            "yearFrom": "2018-12-31T23:00:00.000Z",
            "yearTo":   "2025-12-31T22:59:59.999Z",
        },
        "operationName": "playerRankingHistory",
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()["data"]["list"]
    if len(data) == 0:
        continue

    latest = data[0]  # most recent record
    all_players = pd.concat(
        [
            all_players,
            pd.DataFrame(
                [
                    {
                        "id": adv_id,
                        "name": f"{latest.get('firstname','').strip()} "
                                f"{latest.get('lastname','').strip()}".strip(),
                        "competitionValue": latest["competitionValue"],
                    }
                ]
            ),
        ],
        ignore_index=True,
    )

# ── (optional) persist the updated master list ────────────────────────────────
# all_players.to_csv("updated_all_players.csv", index=False)


# paralellize

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pprint import pprint
import threading
from tqdm import tqdm
import requests, pandas as pd, os, pathlib, json, time

# ------------------------------------------------------------------
# CONSTANTS ─────────────────────────────────────────────────────────
# ------------------------------------------------------------------
BASE_DIR = pathlib.Path("data/singles")
BASE_DIR.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# A helper so every thread re-uses its own connection pool
# ------------------------------------------------------------------
thread_local = threading.local()


def session() -> requests.Session:
    if not hasattr(thread_local, "sess"):
        thread_local.sess = requests.Session()
        thread_local.sess.headers.update(headers)
    return thread_local.sess

missing_ids: set[int] = set()

# ------------------------------------------------------------------
# Core worker executed in parallel
# ------------------------------------------------------------------
def fetch_and_store(player) -> None:
    """Download both datasets for one player and write the parquet."""
    # --- 1) ranking history -------------------------------------------------
    # if player.name!="Severin Habe":
    #     return
    payload_1 = {
        "query":
        "query playerRankingHistory($id: Int!, $yearFrom: timestamp, $yearTo: timestamp) {\n  list: RankingHistory(\n    where: {personId: {_eq: $id}, date: {_lte: $yearTo, _gte: $yearFrom}}\n    order_by: {date: desc}\n  ) {\n    personId\n    date\n    classification\n    rank\n    competitionValue\n    classificationValue\n    games\n  }\n  currentRanking: lizenz_nehmer(where: {personId: {_eq: $id}}) {\n    ranking\n    classification\n    classificationValue\n    competitionValue\n    numberOfGames\n  }\n  currentSeason: RankSeasonRange(where: {seasonPointer: {_eq: 1}}) {\n    dateBegin\n  }\n  previousSeason: RankSeasonRange(where: {seasonPointer: {_eq: 0}}) {\n    dateBegin\n  }\n}",
        "variables": {
            "id": player.id,
            "yearFrom": "2018-12-31T23:00:00.000Z",
            "yearTo": "2025-12-31T22:59:59.999Z"
        },
        "operationName": "playerRankingHistory"
    }
    s = session()
    r1 = s.post(url, json=payload_1, timeout=30)
    r1.raise_for_status()
    history = r1.json()["data"]["list"]
    if len(data) == 0:
        return
    competition_prev = history[0]["competitionValue"]

    # --- 2) single results ---------------------------------------------------
    payload_2 = {
        "query":
        "query SingleResults($where: AllSingleResults_bool_exp, $orderBy: [AllSingleResults_order_by!], $limit: Int = 300) {\n  results: AllSingleResults(where: $where, order_by: $orderBy, limit: $limit) {\n    ...singleResult\n  }\n}\n\nfragment singleResult on AllSingleResults {\n  date\n  tournamentName\n  adversaryPersonId\n  adversaryFirstname\n  adversaryLastname\n  playerSet1WonGames\n  adversarySet1WonGames\n  playerSet2WonGames\n  adversarySet2WonGames\n  playerSet3WonGames\n  adversarySet3WonGames\n  playerSet4WonGames\n  adversarySet4WonGames\n  playerSet5WonGames\n  adversarySet5WonGames\n  playerWinnerCode\n  Asr_AdversaryValue_f\n  adversary {\n    classification\n    classificationValue\n    competitionValue4L\n    ranking\n  }\n  ATPWTARank\n  Asr_Id_Item_l\n  round: Asr_Round_i\n  encounterId: Asr_Id_Item_l\n  matchNotConsidered\n  event {\n    id\n    tournamentId\n    eventModeId\n  }\n  source\n  rank: Asr_RankConvert_s\n}",
        "variables": {
            "where": {
                "playerPersonId": {
                    "_eq": player.id
                },
                "playerWinnerCode": {
                    "_neq": "D"
                },
                "source": {
                    "_in": [
                        310, 311, 110, 111, 120, 121, 210, 211, 220, 221, 250, 150, 410, 411, 420, 421, 450, 500, 501,
                        600, 601, 660, 610, 650, 700, 701
                    ]
                },
                "rankSeasonRange": {
                    "seasonPointer": {
                        "_in": [-1, 0]
                    }
                }
            },
            "orderBy": {
                "date": "desc"
            }
        },
        "operationName": "SingleResults"
    }
    r2 = s.post(url, json=payload_2, timeout=30)
    r2.raise_for_status()
    results = r2.json()["data"]["results"]
    # --- 3) normalise -> filter -> save -------------------------------------
    df = pd.json_normalize(results)
    if df.empty:
        outfile = (BASE_DIR / f"{player.name.replace(' ','*')}_competitionValuePrev={competition_prev}"
                   f"__competitionValue={player.competitionValue}.parquet")
        df.to_parquet(outfile, index=True)
        return

    # keep only the columns you care about
    cols_keep = [
        "date",
        "adversaryFirstname",
        "adversaryLastname",
        "adversaryPersonId",
        "playerWinnerCode",
        "adversary.classification",
        "rank",
        "adversary.competitionValue4L",
        "Asr_AdversaryValue_f",
    ]
    df = df[[c for c in cols_keep if c in df.columns]]

    df["date"] = pd.to_datetime(df["date"], format='mixed')
    df = df.set_index("date").sort_index(ascending=False)

    # fill missing values exactly like in your original
    if "adversary.classification" in df.columns:
        df["adversary.classification"] = df["adversary.classification"].fillna(df["rank"])

    if "adversary.competitionValue4L" not in df.columns:
        df["adversary.competitionValue4L"] = df["Asr_AdversaryValue_f"]
    else:
        df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(df["Asr_AdversaryValue_f"])

    df = df[df["adversary.competitionValue4L"] != 0]
    
    if "adversaryPersonId" in df.columns:
        missing_ids.update(
            set(df["adversaryPersonId"].dropna().astype(int))
            - set(all_players["id"].astype(int))
        )

    outfile = (BASE_DIR / f"{player.name.replace(' ','*')}_competitionValuePrev={competition_prev}"
               f"__competitionValue={player.competitionValue}.parquet")
    df.to_parquet(outfile, index=True)


# ------------------------------------------------------------------
# Kick-off with a progress bar
# ------------------------------------------------------------------
MAX_WORKERS = 8  # tune to taste / server limits
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
    futures = {pool.submit(fetch_and_store, p): p for p in all_players.itertuples()}
    for _ in tqdm(as_completed(futures), total=len(futures), desc="players"):
        pass


players: 100%|██████████| 2000/2000 [00:45<00:00, 44.21it/s]


In [9]:
def fetch_ranking(adv_id: int) -> dict | None:
    """Return a dict with id, name, competitionValue or None if not found."""
    payload = {
        "query": 
        "query playerRankingHistory($id: Int!, $yearFrom: timestamp, $yearTo: timestamp) {\n  list: RankingHistory(\n    where: {personId: {_eq: $id}, date: {_lte: $yearTo, _gte: $yearFrom}}\n    order_by: {date: desc}\n  ) {\n    personId\n    firstname\n  lastname\n  date\n    classification\n    rank\n    competitionValue\n    classificationValue\n    games\n  }\n  currentRanking: lizenz_nehmer(where: {personId: {_eq: $id}}) {\n    ranking\n    classification\n    classificationValue\n    competitionValue\n    numberOfGames\n  }\n  currentSeason: RankSeasonRange(where: {seasonPointer: {_eq: 1}}) {\n    dateBegin\n  }\n  previousSeason: RankSeasonRange(where: {seasonPointer: {_eq: 0}}) {\n    dateBegin\n  }\n}",
        "variables": {
            "id": adv_id,
            "yearFrom": "2018-12-31T23:00:00.000Z",
            "yearTo":   "2025-12-31T22:59:59.999Z",
        },
        "operationName": "playerRankingHistory",
    }
    try:
        resp = requests.post(url, json=payload, headers=headers, timeout=30)
        resp.raise_for_status()
    except Exception:
        return None
    data = resp.json()["data"]["list"]
    if not data:
        return None

    latest = data[0]
    return {
        "id": adv_id,
        "name": f"{latest['firstname']} "
                f"{latest['lastname']}".strip(),
        "competitionValue": latest["competitionValue"],
    }

new_rows: list[dict] = []

with ThreadPoolExecutor(max_workers=min(32, len(missing_ids) or 1)) as pool:
    futures = {pool.submit(fetch_ranking, i): i for i in missing_ids}
    for _ in tqdm(as_completed(futures), total=len(futures),
                  desc="Fetching missing adversaries"):
        res = _.result()
        if res:
            new_rows.append(res)

if new_rows:  # append to all_players in one go
    all_players = pd.concat(
        [all_players, pd.DataFrame(new_rows)],
        ignore_index=True,
    )


Fetching missing adversaries: 100%|██████████| 6116/6116 [04:26<00:00, 22.97it/s]


In [10]:
all_players

,id,name,competitionValue
0,70318,Stan Wawrinka,15.299
1,126520,Marc-Andrea Hüsler,14.816
2,172242,Jérôme Kym,14.604
3,122182,Alexander Ritschard,14.644
4,158613,Dominic Stricker,14.558
...,...,...,...
8111,32761,,6.040
8112,19841013,,2.884
8113,19824638,,4.371
8114,19808246,,3.934


In [111]:
data=response.json()["data"]["results"]
df = pd.json_normalize(data)
df = df[[
    "date",
    "adversaryFirstname",
    "adversaryLastname",
    "playerWinnerCode",
    "adversary.classification",
    "rank",
    "adversary.competitionValue4L",
    "Asr_AdversaryValue_f",
]]
df["date"] = pd.to_datetime(df["date"])
df=df.set_index("date").sort_index(ascending=False)
df["adversary.classification"] = df["adversary.classification"].fillna(df["rank"])
df["adversary.competitionValue4L"] = df["adversary.competitionValue4L"].fillna(df["Asr_AdversaryValue_f"])
df = df[[
    "adversaryFirstname",
    "adversaryLastname",
    "playerWinnerCode",
    "adversary.classification",
    "adversary.competitionValue4L",
]]
df


,adversaryFirstname,adversaryLastname,playerWinnerCode,adversary.classification,adversary.competitionValue4L
date,,,,,
2025-05-24 12:00:00,Alejo,Prado,N,N3,11.728000
2025-05-10 12:00:00,Noël,Kunz,S,R2,8.220000
2025-05-03 12:00:00,Björn,Schilter,N,R2,8.069000
2025-04-26 00:00:00,William,Kjellberg,N,R1,9.207924
2025-04-26 00:00:00,Paolo,Iallonardo,S,R1,9.098000
2025-04-26 00:00:00,Daniele,Chiappini,S,R2,8.062016
2025-04-19 00:00:00,Benjamin,Freyholdt,S,N4,9.764000
2025-04-19 00:00:00,Felix,Seres,N,N4,11.061000
2025-04-19 00:00:00,Petr,Minakhin,S,R1,9.207924
